In [ ]:
from BiotSolver import solve_biot
from fenics import *
from mshr import *
from SimpleBrainMesh import generate_doughnut_mesh
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *


T = 1.2            # final time
num_steps = 24     # number of time steps
dt = T / num_steps # time step size


kappa = 15*1e-12     # permeability 15*(1e-9)**2
visc = 0.8*1e-3          # viscocity 
K = Constant(kappa/visc) # hydraulic conductivity
E = 1500.0 # Young modulus
nu = 0.49999 # Poisson ratio

c = 2*1e-4  # storage coefficent
alpha = 0.5 # Biot-Willis coefficient (change of the bulk volume due to pore pressure change)


# Lame coefficients
lmbda = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
mu = E/(2.0*(1.0+nu))

mmHg2Pa = 133.32
material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K 
material_parameter["lambda"] = lmbda
material_parameter["mu"] = mu
material_parameter["alpha"] = alpha


# create mesh
size = 0.1
N = 15
mesh = RectangleMesh(Point(0.0, 0.0), Point(size, size), N, N)


ventricle = CompiledSubDomain("on_boundary && near(x[0], 0.0)")
skull = CompiledSubDomain("on_boundary && near(x[0], size)", size=size)

p_D_skull = Constant(0.0)
p_D_ventricle = Expression("a*sin(2*pi*f*( t + d) )", t=0, f=1, a=5*mmHg2Pa, d=0.0, degree=2)
#p_D_ventricle = Constant(1.0)
u_D_skull = Constant([0.0, 0.0])

probe_points = [
                Point(0.0, 0.5*size),
                Point(0.2*size, 0.5*size),
                Point(0.4*size, 0.5*size),
                ]

y = 0.5*size
x_coords = np.linspace(0.0, size, 1000)
slice_points = [Point(x,y) for x in x_coords]

In [ ]:
results = solve_biot(mesh, material_parameter, T, num_steps,
                     skull, ventricle, 
                     p_D_skull=p_D_skull,
                     p_D_ventricle=p_D_ventricle,
                     u_D_skull=u_D_skull,
                     probe_points=probe_points,
                     slice_points=slice_points)

In [ ]:
times = [ 0.2, 0.4, 0.6, 0.8, 1.0]

idx = []
for i in range(num_steps):
    if np.isclose((i +1)*dt, times).any():
        idx.append(i)
cross_section = results["p_fluid_slice"]
plt.figure(figsize=(10,10))
for i in idx:
    plt.plot(x_coords, cross_section[i,:], label="t = {}".format((i +1)*dt))
plt.legend()
plt.grid()
plt.xlabel("x in m")
plt.ylabel("p in Pa")


In [ ]:
p_data = results["probe_point_p_fluid"]
plt.figure(figsize=(10,10))
num_points = len(probe_points)
for i in range(num_points):
    plt.plot(np.linspace(0,T, num_steps), 
             p_data[:,i], 
             label="$P_{}$".format(i)
            )
plt.xlabel("t in s")
plt.ylabel("p in Pa")
plt.grid()
plt.legend()


In [ ]:
plt.figure(figsize=(8,8))
cont = plot(mesh)
for i, point in enumerate(probe_points):
    plt.scatter(point.array()[0], point.array()[1],
                marker=".", s= 300)
    plt.annotate( "$P_{}$".format(i), point.array()[0:2], size=20)